<h1>Testing the numerical matrix generation</h1>

## -- HiCAT --

Testing the numerical mlatrix generation with the HiCAT simulator.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from astropy.io import fits
import astropy.units as u

%pylab inline
import poppy
import hicat.simulators

os.chdir('../../pastis/')
from config import CONFIG_INI
import util_pastis as util

## Testing the code in form of a notebook

In [ ]:
# Parameters
# Nothing to read. I will do all of it locally.
nb_seg = 36

## Basic hicat simulator and create a PSF

In [ ]:
hicat = hicat.simulators.hicat_sim.HICAT_Sim()
print('HiCAT simulator')

The simulator mmics the `hicat` repo's configfile. I don't want that now, so I will have to set all components manually.

First, lets view the current simulator's state.

In [ ]:
hicat.testbed_state

In [ ]:
# Set the components I want
hicat.iris_ao = 'iris_ao'
hicat.apodizer = 'no_apodizer'
hicat.lyot_stop = 'circular'

In [ ]:
# Get some more info
hicat.ref_info['fpm']

In [ ]:
# Get some more info
hicat.ref_info['lyot_stop']

In [ ]:
# Get the full HiCAT sim info one more time
print(hicat.describe())

Lets create a PSF now.

In [ ]:
plt.figure(figsize=(15, 15))
psf, waves = hicat.calc_psf(display=True, return_intermediates=True)

The `waves` variable holds arrays of the E-field at each of the 12 planes.

In [ ]:
plt.imshow(waves[0].phase)

Add an apodizer and the according Lyot stop and recalculate the PSF.

In [ ]:
hicat.apodizer = 'cnt1_apodizer'
hicat.lyot_stop = 'cnt1_apodizer_lyot_stop'
print(hicat.describe())

In [ ]:
plt.figure(figsize=(15, 15))
psf2, waves2 = hicat.calc_psf(display=True, return_intermediates=True)

In [ ]:
plt.figure(figsize=(10,8))
poppy.display_psf(psf2, vmin=1e-10, vmax=1e-5) # NOT contrast units
plt.title("PSF with apodizer")

In [ ]:
# Extract the PSF array
psf_ar = psf2[0].data

In [ ]:
plt.imshow(psf_ar, vmin=1e-10, vmax=1e-7)

### DH mask

I need to get the image in the DH only, and for that I need a DH mask. I want to be able to tie that directly to $lambda/D$ in the focal plane, but because I don't know how to do that right, now I'll wing it by plugging in an estimated sampling. THe samplling will change based on how mich the images are binned, which is refelcted inthe configfile.

In [ ]:
dh_mask = util.create_dark_hole(psf_ar, iwa=5, owa=12, samp = 13)
plt.imshow(dh_mask)
plt.title('dh_mask')

In [ ]:
test = psf_ar * dh_mask

plt.figure()
plt.imshow(test, vmin=1e-10, vmax=1e-8)
plt.title('DH of HiCAT image')

Looking good.

### Create a direct E2E PSF witih apodizer to obtain normalization factor

No coronagraph, no aberrations. No coronagraph for the HiCAT case means no FPM, but the apodizer and according Lyot stop stay in, as does the Iris AO. I don't remember how to take direct images in the simulator. Do I just move the FPM out of the beam? Edit: Yes, with `hc.include_fpm = False`.

In [ ]:
# Set up the direct imaging mode for the HiCAT simulator
hicat.include_fpm = False
print(hc.describe())

In [ ]:
# Calculate PSF
psf_hc = hicat.calc_psf(display=False, return_intermediates=False)
psf_perfect = psf_hc[0].data

# Normalize PSF
normp = np.max(psf_perfect)
psf_perfect = psf_perfect / normp

# Show PSF
plt.figure(figsize=(15, 15))
plt.imshow(psf_perfect, norm=LogNorm())
plt.title('Direct PSF for normalization purposes')
plt.colorbar()
plt.show()

print('PSf shape:', psf_perfect.shape)
print('PSF max:', np.max(psf_perfect))

### Set HiCAT up with the coronagraph

Put the apodizer in, swap in the Lyot stop that goes with the apodizer and put the FPM back in.

In [ ]:
# Set up HiCAT with the coronagraph
hc.apodizer = 'cnt1_apodizer'
hc.lyot_stop = 'cnt1_apodizer_lyot_stop'
hc.include_fpm = True
print(hc.describe())

## Control the Iris AO DM

I can control the segments individually in PTT.

In [ ]:
hicat.iris_dm.flatten()
hicat.iris_dm.set_actuator(35, 200*u.nm, 0, 0)  # adjust piston (m) without adding tip or tilt

In [ ]:
# Display the pistoned segment in all planes
plt.figure(figsize=(14,14))
psf = hicat.calc_psf(display=True)

We definitely get some extra structure in the dark hole from that. I looped thorugh all 37 segments to confirm I know they're numbered all right.

Now lets do a pair of pistoned segments.

### Aberrating pairs of segments on the Iris AO

In [ ]:
seg1 = 3
seg2 = 6
aber = 0 * u.nm

hicat.iris_dm.flatten()
hicat.iris_dm.set_actuator(seg1, aber, 0, 0)  # adjust piston (m) without adding tip or tilt
hicat.iris_dm.set_actuator(seg2, aber, 0, 0)  # adjust piston (m) without adding tip or tilt

In [ ]:
# Display the pistoned segment in all planes
plt.figure(figsize=(14,14))
psf, pair_waves = hicat.calc_psf(display=True, return_intermediates=True)

In [ ]:
# Display the DH only
psf_pair = psf[0].data/normp * dh_mask

print('Aberration: {}'.format(aber))
print('On segments {} and {}.'.format(seg1, seg2))

plt.imshow(psf_pair, vmax=1e-5)

In [ ]:
# Do I know how to claculate the contrast?
c_test = util.dh_mean(psf[0].data/normp, dh_mask)
print(c_test)

Can I display the OPD on the IrisAO and save that?

In [ ]:
# This gives me all the planes
plt.figure(figsize=(8, 25))
hc.display()

In [ ]:
# This will display the phase in the third plane, the phase after the apodizer.
plt.imshow(pair_waves[2].phase)

It doesn't really give me an outline of the IrisAO so it's a bit hard to tell which segments are aberrated, but if I have many of these in sequence, it will be very easy to figure that out.

## Generate numerical PASTIS matrix

Next, making the loop to generate one image per aberrated segment pair i, j and feeding the resulting contrast into the matrix element [i, j].

In [ ]:
#-# Generating the PASTIS matrix
#nb_seg = 36
matrix_direct = np.zeros([nb_seg, nb_seg])   # Generate empty matrix
aber = 100*u.nm

print('aber: {}'.format(aber))

# List for saving PSFs and DHs and contrasts
all_psfs = []
all_dhs = []
all_contrasts = []

for i in range(nb_seg):
    for j in range(nb_seg):

        print('STEP: {}-{} / {}-{}'.format(i+1, j+1, nb_seg, nb_seg))

        # Putting aberrations on segments i and j
        hicat.iris_dm.flatten()
        
        # Apply both aberrations to OTE. If i=j, apply only once!
        hicat.iris_dm.set_actuator(i+1, aber, 0, 0)    # set segment i (segment numbering starts at 1)
        if i != j:
            hicat.iris_dm.set_actuator(j+1, aber, 0, 0)    # set segment j
        
        print('Calculating HiCAT sim image')
        image, opds = hicat.calc_psf(display=False, return_intermediates=True)
        psf = image[0].data/normp
        
        # Save OPD images of the IrisAO
        iris_opd = opds[2].phase
        opd_name = 'opd_piston_Noll1_segs_' + str(i+1) + '-' + str(j+1)
        plt.clf()
        plt.imshow(iris_opd)
        plt.savefig(os.path.join('/Users/ilaginja/Documents/Git/PASTIS/Jupyter Notebooks/HiCAT', 'OTE_images', opd_name))
        #plt.show()
        
        # Save HiCAT image to disk
        filename_psf = 'psf_' + 'piston_Noll1_segs_' + str(i+1) + '-' + str(j+1)
        util.write_fits(psf, os.path.join('/Users/ilaginja/Documents/Git/PASTIS/Jupyter Notebooks/HiCAT', 'psfs', filename_psf + '.fits'), header=None, metadata=None)
        all_psfs.append(psf)
        
        print('Calculating mean contrast in dark hole')
        dh_intensity = psf * dh_mask
        contrast = np.mean(dh_intensity[np.where(dh_intensity != 0)])
        print('contrast:', contrast)
        
        # Save DH image to disk and put current contrast in list
        filename_dh = 'dh_piston_Noll1_segs_' + str(i+1) + '-' + str(j+1)
        util.write_fits(dh_intensity, os.path.join('/Users/ilaginja/Documents/Git/PASTIS/Jupyter Notebooks/HiCAT', 'darkholes', filename_dh + '.fits'), header=None, metadata=None)
        all_dhs.append(dh_intensity)
        all_contrasts.append(contrast)
        
        # If you want to display the PSF and DH image
        plt.figure(figsize=(15,10))
        plt.subplot(1, 2, 1)
        plt.imshow(psf, norm=LogNorm(), origin='lower')
        plt.colorbar()
        plt.title('PSF {}/{}'.format(i+1, j+1))
        plt.subplot(1, 2, 2)
        plt.imshow(dh_intensity, norm=LogNorm(), origin='lower')
        plt.colorbar()
        plt.title('DH image {}/{}'.format(i+1, j+1))
        plt.show()

        # Fill according entry in the matrix
        matrix_direct[i,j] = contrast
        
all_psfs = np.array(all_psfs)
all_dhs = np.array(all_dhs)
all_contrasts = np.array(all_contrasts)

print("All done")

In [ ]:
# Print the contrast values
print('Measured mean contrasts in the dark holes, per single aberrated segment:')
print(all_contrasts)
print('Mean:', np.mean(all_contrasts))
print('Min:', np.min(all_contrasts))
print('Max:', np.max(all_contrasts))

In [ ]:
# You can flick through the images here
imnum1 = 6

# Figure out what segment pair the number "imnum" corresponds to
seg1 = int(np.floor(imnum1/nb_seg)) + 1
seg2 = imnum1%nb_seg + 1

plt.figure(figsize=(18, 9))
plt.suptitle('Segment pair:' + str(seg1) + '-' + str(seg2))
plt.subplot(1, 2, 1)
plt.imshow(all_psfs[imnum1], norm=LogNorm(), origin='lower')
plt.title('Whole PSF')
plt.subplot(1, 2, 2)
plt.imshow(all_dhs[imnum1], norm=LogNorm(), origin='lower')
plt.title('Dark hole')
plt.show()

In [ ]:
# Testing what area the mean is taken of
test_dh = np.copy(all_dhs[imnum1])
test_dh[np.where(test_dh != 0)] = 10000

zoomim = 50

plt.figure(figsize=(18, 9))
plt.subplot(1, 2, 1)
plt.imshow(util.zoom_cen(all_dhs[imnum1], zoomim), norm=LogNorm(), origin='lower')
plt.title('DH image')
plt.subplot(1, 2, 2)
plt.imshow(util.zoom_cen(dh_mask, zoomim), origin='lower')
plt.title('Marked DH')
plt.show()

In [ ]:
count_test = np.count_nonzero(dh_mask)
count_dh = np.count_nonzero(all_dhs[imnum1])
print('Number of pixels in marked array:', count_test)
print('Number of pixels in DH array:', count_dh)
print('Mean in marked array:', np.sum(test_dh)/count_test)
print('Mean in DH array:', np.sum(all_dhs[imnum1])/count_dh)

In [ ]:
# Display the matrix
plt.imshow(matrix_direct)
plt.title('Numerical matrix')
plt.show()

In [ ]:
print(matrix_direct)

## Calculating the off-axis elements

Analogous to the off-axis elements in the analytical matrix (see notebook: **6_Testing analytical matrix generation**), we need to correct the off-axis elements.

In [ ]:
# Filling the off-axis elements
matrix_two_N = np.copy(matrix_direct)      # This is just an intermediary copy so that I don't mix things up.
matrix_pastis = np.copy(matrix_direct)     # This will be the final PASTIS matrix.

for i in range(nb_seg):
    for j in range(nb_seg):
        if i != j:
            matrix_off_val = (matrix_two_N[i,j] - matrix_two_N[i,i] - matrix_two_N[j,j]) / 2.
            matrix_pastis[i,j] = matrix_off_val
            print('Off-axis for i{}-j{}: {}'.format(i+1, j+1, matrix_off_val))

In [ ]:
# Normalize matrix for the input aberration
matrix_pastis /= np.square(aber.value)

Save the matrix.

In [ ]:
# Save matrix to file
where_to = '/Users/ilaginja/Documents/Git/PASTIS/Jupyter Notebooks/HiCAT'
filename_matrix = 'PASTISmatrix_num_HiCAT_piston_Noll1'
util.write_fits(matrix_pastis, os.path.join(where_to, filename_matrix + '.fits'), header=None, metadata=None)
print('Matrix saved to:', os.path.join(where_to, filename_matrix + '.fits'))

## Displaying the results from the integrated script on the repo

### *-- Can't do this for HiCAT yet --*

In [ ]:
# Read the matrix
filename_matrix = 'PASTISmatrix_num_' + zern_mode.name + '_' + zern_mode.convention + str(zern_mode.index)
matrix = fits.getdata(os.path.join(resDir, filename_matrix + '.fits'))

# Read the PSFs
psf_cube = fits.getdata(os.path.join(resDir, 'psfs', 'psf_cube' + '.fits'))

# Read the DH images
dh_cube = fits.getdata(os.path.join(resDir, 'darkholes', 'dh_cube' + '.fits'))

# Read the contrasts
con = np.loadtxt(os.path.join(resDir, 'contrasts.txt'))

In [ ]:
# Display numerical matrix
plt.figure(figsize=(20, 20))
plt.imshow(matrix, origin='lower')
plt.show()

In [ ]:
# You can flick through the images here
nb_seg = 18   # this has to be the same number like in the script you ran
imnum2 = 200

# Figure out what segment pair the number "imnum" corresponds ro
seg1 = int(np.floor(imnum2/nb_seg)) + 1
seg2 = imnum2%nb_seg + 1

plt.figure(figsize=(18, 9))
plt.suptitle('Segment pair:' + str(seg1) + '-' + str(seg2))
plt.subplot(1, 2, 1)
plt.imshow(psf_cube[imnum2], norm=LogNorm(), origin='lower')
plt.title('Whole PSF')
plt.subplot(1, 2, 2)
plt.imshow(dh_cube[imnum2], norm=LogNorm(), origin='lower')
plt.title('Dark hole')
plt.show()